# Step 3-2a. 이미지 임베딩 데이터 업로드 (Titan - 영어 전용)
Bedrock Titan 멀티모달 모델을 사용하여 이미지를 벡터로 변환하고 OpenSearch에 업로드합니다.

## 멀티모달 임베딩이란?
- **텍스트와 이미지를 같은 벡터 공간**에 임베딩할 수 있는 모델
- `amazon.titan-embed-image-v1` 모델은 이미지와 텍스트 모두를 1024차원 벡터로 변환
- 이를 통해 **텍스트로 이미지 검색**, **이미지로 이미지 검색**이 모두 가능

## 주의: 영어만 지원
- Titan 멀티모달 모델은 **영어 텍스트에만 최적화**되어 있습니다
- 한국어 텍스트로 검색하면 유사도 점수가 모두 비슷하게 나와 구분이 어렵습니다
- 한국어 지원이 필요하면 **Step 3-2b (Nova)** 노트북을 사용하세요

## 실습 과제
- 다양한 이미지를 `data/image` 경로에 넣어서 테스트해보세요
- 용량이 너무 큰 파일은 안되니 **10MB 이하**로 실습

In [ ]:
!pip install -q boto3==1.38.46 opensearch-py==2.8.0

## 1. 설정 (Configuration)

In [ ]:
import os, json

# Step 0에서 저장한 설정 불러오기
try:
    with open("../config.json") as f:
        _config = json.load(f)
    print("✅ config.json 로드 완료")
except FileNotFoundError:
    raise FileNotFoundError("❌ config.json을 찾을 수 없습니다. Step 0 노트북을 먼저 실행해주세요.")

HOST = _config.get("OPENSEARCH_HOST")
if not HOST:
    raise ValueError("❌ config.json에 OPENSEARCH_HOST 값이 없습니다. Step 0 노트북을 먼저 실행해주세요.")
DEFAULT_REGION = _config.get("DEFAULT_REGION", "ap-northeast-2")
BEDROCK_REGION = _config.get("BEDROCK_REGION", "us-east-1")
PROFILE = _config.get("PROFILE", "skku-opensearch-session")

INDEX_NAME = 'titan-image-test'
IMAGE_DIRECTORY_PATH = '../../data/image'
EMBEDDING_MODEL_ID = 'amazon.titan-embed-image-v1'
VECTOR_DIMENSION = 1024

## 2. OpenSearch 클라이언트 생성

In [2]:
import boto3
from opensearchpy import OpenSearch, AWSV4SignerAuth, RequestsHttpConnection

service = 'aoss'
credentials = boto3.Session(profile_name=PROFILE).get_credentials()
auth = AWSV4SignerAuth(credentials, DEFAULT_REGION, service)

client = OpenSearch(
    hosts=[{'host': HOST, 'port': 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

print("OpenSearch 클라이언트 생성 완료")

OpenSearch 클라이언트 생성 완료


## 3. Bedrock 클라이언트 생성 및 이미지 임베딩 함수 정의

In [ ]:
import json
import base64
import math

print(f"Creating a boto3 session with profile '{PROFILE}'...")
session = boto3.Session(profile_name=PROFILE)

print(f"Creating a Bedrock client in region: {BEDROCK_REGION}")
bedrock_client = session.client(
    service_name='bedrock-runtime',
    region_name=BEDROCK_REGION,
)
print("Bedrock client created successfully.")

def normalize_vector(vec):
    """벡터를 L2 정규화합니다. innerproduct에서 코사인 유사도와 동일한 결과를 얻기 위함."""
    norm = math.sqrt(sum(x * x for x in vec))
    return [x / norm for x in vec] if norm > 0 else vec

def image_to_base64(image_path):
    """이미지 파일 경로를 받아 Base64로 인코딩된 문자열을 반환합니다."""
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"Error: Image file not found at {image_path}")
        return None
    except Exception as e:
        print(f"Error encoding image {image_path}: {e}")
        return None

def get_image_embedding_from_bedrock(base64_image_data, model_id):
    """Bedrock API를 호출하여 Base64 이미지 데이터의 벡터 임베딩을 반환합니다."""
    body = json.dumps({"inputImage": base64_image_data})
    response = bedrock_client.invoke_model(
        body=body,
        modelId=model_id,
        accept="application/json",
        contentType="application/json",
    )
    response_body = json.loads(response.get("body").read())
    return normalize_vector(response_body.get("embedding"))

## 4. 이미지 벡터 인덱스 생성 (기존 데이터 삭제)
인덱스가 이미 존재하면 기존 문서를 모두 삭제하고, 존재하지 않으면 새로 생성합니다.

In [ ]:
index_body = {
    "settings": {
        "index": {
            "knn": True,
        }
    },
    "mappings": {
        "properties": {
            "content_vector": {
                "type": "knn_vector",
                "dimension": VECTOR_DIMENSION,
                "method": {
                    "name": "hnsw",
                    "space_type": "innerproduct",
                    "engine": "faiss"
                }
            },
            "image_path": {"type": "keyword"},
        }
    }
}

if client.indices.exists(index=INDEX_NAME):
    doc_count = client.count(index=INDEX_NAME)['count']
    if doc_count > 0:
        print(f"🗑️ '{INDEX_NAME}' 인덱스에서 기존 문서 {doc_count}개 삭제 중...")
        client.delete_by_query(index=INDEX_NAME, body={"query": {"match_all": {}}})
        print(f"✅ 기존 문서 삭제 완료")
    else:
        print(f"✅ '{INDEX_NAME}' 인덱스가 이미 존재하며 비어있습니다.")
else:
    print(f"📦 '{INDEX_NAME}' 인덱스 생성 중...")
    client.indices.create(index=INDEX_NAME, body=index_body)
    print(f"✅ 인덱스 생성 완료")

## 5. 이미지 임베딩 및 업로드

In [8]:
from opensearchpy import helpers

print(f"Loading image files from '{IMAGE_DIRECTORY_PATH}'...")
image_files = [f for f in os.listdir(IMAGE_DIRECTORY_PATH) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
print(f"Found {len(image_files)} image files.")

def generate_bulk_actions(img_files, img_dir):
    for i, file_name in enumerate(img_files):
        print(f"  - Processing image {i+1}/{len(img_files)}: {file_name}")
        full_path = os.path.join(img_dir, file_name)

        base64_data = image_to_base64(full_path)
        if not base64_data:
            continue

        vector = get_image_embedding_from_bedrock(base64_data, EMBEDDING_MODEL_ID)
        if not vector:
            print(f"Warning: Could not generate embedding for {file_name}. Skipping.")
            continue

        source_data = {
            "content_vector": vector,
            "image_path": full_path,
        }

        yield {
            "_index": INDEX_NAME,
            "_source": source_data
        }

print("\nStarting image embedding and uploading via Bedrock...")
success, failed = helpers.bulk(client, generate_bulk_actions(image_files, IMAGE_DIRECTORY_PATH))

print(f"\nSuccessfully indexed {success} images.")
if failed:
    print(f"Failed to index {len(failed)} images.")
    for i, item in enumerate(failed[:5]):
        print(f"Failed item {i+1}: {item}")

Loading image files from '../../data/image'...
Found 12 image files.

Starting image embedding and uploading via Bedrock...
  - Processing image 1/12: bee_1.png
  - Processing image 2/12: bee_2.jpg
  - Processing image 3/12: bee_3.jpg
  - Processing image 4/12: wasp_3.jpg
  - Processing image 5/12: cat_1.jpg
  - Processing image 6/12: wasp_2.jpg
  - Processing image 7/12: cat_2.jpg
  - Processing image 8/12: cat_3.jpg
  - Processing image 9/12: wasp_1.png
  - Processing image 10/12: dog_1.png
  - Processing image 11/12: dog_3.jpg
  - Processing image 12/12: dog_2.jpg

Successfully indexed 12 images.
